In [18]:
#导入包
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets,transforms
from torch.utils.data import DataLoader

In [19]:
#构建transforms

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,),(0.3081,))
])

In [20]:
#下载，加载数据集

trainset = datasets.MNIST('data',train = True, download = True, transform = transform)
testset = datasets.MNIST('data',train = False, download = True, transform = transform)

device = 'cuda'
BATCH_SIZE = 64

train_loader = DataLoader(trainset, batch_size = BATCH_SIZE, shuffle = True, num_workers = 0, pin_memory = True)
test_loader = DataLoader(testset, batch_size = BATCH_SIZE, shuffle = True, num_workers = 0, pin_memory = True)

In [21]:
#构建网络模型

class CNN_Net(nn.Module):
    def __init__(self):
        super(CNN_Net,self).__init__()
        #图片 1*28*28
        self.conv1 = nn.Conv2d(1,6,5) #24*24*20
        self.pool = nn.MaxPool2d(2,2) # 12*12*20
        self.conv2 = nn.Conv2d(6,16,3)# 10*10*40
        #5*5*40
        self.fc1 = nn.Linear(5*5*16,120)
        self.fc2 = nn.Linear(120,84)
        self.fc3 = nn.Linear(84,10)
    def forward(self,x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        
        x = x.view(-1,5*5*16)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
#创建模型
net = CNN_Net().to(device)

In [22]:
#定义优化器和损失函数

criterion = nn.CrossEntropyLoss() # 交叉式损失函数
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9) # 优化器

In [23]:
#训练模型

EPOCH = 10

for epoch in range(EPOCH):
    train_loss = 0.0
    for i,(datas,labels) in enumerate(train_loader):
        #数据处理
        datas = datas.to(device)
        labels = labels.to(device)
        #梯度置零
        optimizer.zero_grad()
        #训练
        outputs = net(datas)
        #计算损失
        loss = criterion(outputs,labels)
        #反向传播
        loss.backward()
        #参数更新
        optimizer.step()
        
        train_loss += loss.item()
    
    print("Epoch :%d , Loss : %.5f" %(epoch+1, train_loss/len(train_loader.dataset)))

Epoch :1 , Loss : 0.00513
Epoch :2 , Loss : 0.00111
Epoch :3 , Loss : 0.00080
Epoch :4 , Loss : 0.00062
Epoch :5 , Loss : 0.00051
Epoch :6 , Loss : 0.00044
Epoch :7 , Loss : 0.00037
Epoch :8 , Loss : 0.00032
Epoch :9 , Loss : 0.00026
Epoch :10 , Loss : 0.00022


In [24]:
# 保存模型
PATH = './mnist_cnn_net.pth'

torch.save(net.state_dict(), PATH)

In [25]:
# 加载模型

model = CNN_Net()

model.load_state_dict(torch.load(PATH)) 

<All keys matched successfully>

In [26]:
# 测试

correct = 0
total = 0
with torch.no_grad():
    for i , (datas, labels) in enumerate(test_loader):
        
        outputs = model(datas)
        _, predicted = torch.max(outputs.data, dim=1) # 第一个是值的张量，第二个是序号的张量
        total += labels.size(0)  
        correct += (predicted == labels).sum()
    print("Accuracy：{:.3f}%".format(correct / total * 100))

Accuracy：99.040%
